In [20]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
# import umap
import umap.umap_ as umap
from collections import defaultdict

from scipy import linalg, spatial
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer, TfidfVectorizer)

from sklearn.utils.extmath import randomized_svd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [21]:
filename =  '/Users/srijeetabiswas/Desktop/Desktop/transfer to mac/torsha/grad school/Fall Semester/courses/AML/Project/clustered_books.csv'
df = pd.read_csv(filename)
(df.head(10))

,title,description,processed_description,cluster
0,The Old Man and the Sea,Librarian's note: An alternate cover edition c...,librarian note alternate cover edition found h...,3
1,The Vampire Lestat,Lestat. The vampire hero of Anne Rice's enthra...,lestat vampire hero anne rice enthralling new ...,1
2,The Poisonwood Bible,The Poisonwood Bible is a story told by the wi...,poisonwood bible story told wife four daughter...,0
3,Different Seasons,Includes the stories “The Body” and “Rita Hayw...,includes story body rita hayworth shawshank re...,0
4,Invisible Man,First published in 1952 and immediately hailed...,first published immediately hailed masterpiece...,3
5,Battle Royale,Koushun Takami's notorious high-octane thrille...,koushun takami notorious high octane thriller ...,0
6,I'll Give You the Sun,"At first, Jude and her twin brother Noah, are ...",first jude twin brother noah inseparable noah ...,5
7,Because of Winn-Dixie,"The summer Opal and her father, the preacher, ...",summer opal father preacher move naomi florida...,9
8,Lover Avenged,Rehvenge has always kept his distance from the...,rehvenge always kept distance brotherhood even...,9
9,Ficciones,The seventeen pieces in Ficciones demonstrate ...,seventeen piece ficciones demonstrate whirlwin...,3


In [22]:
df['processed_description'] = df['processed_description'].fillna('')

Split Train and Validation Set

In [59]:
from sklearn.model_selection import train_test_split
training, development = train_test_split(df, test_size=0.3, random_state=42)

In [60]:
y_train, y_dev = training['cluster'], development['cluster']

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, min_df=2)
training_text = training['processed_description']
development_text = development['processed_description']

X_train_bow = vectorizer.fit_transform(training_text)
X_development_bow = vectorizer.transform(development_text)

print('Number of unigram', len(vectorizer.get_feature_names_out()))

Number of unigram 21605


In [62]:
# Sum the counts of each bigram across all documents
unigram_counts = X_train_bow.sum(axis=0)
unigram_counts = np.array(unigram_counts).flatten()


# Get sorted indices based on counts (in descending order)
sorted_indices = unigram_counts.argsort()[::-1]

# Retrieve the top 10 unigram and their counts using the indices
top_10_unigram = [(vectorizer.get_feature_names_out()[i], unigram_counts[i]) for i in sorted_indices[:10]]

print("Top 10 unigram:")
for unigram, count in top_10_unigram:
    print(f"'{unigram}': {count}")

Top 10 unigram:
'one': 3046
'life': 3032
'world': 2321
'new': 2214
'year': 1965
'time': 1958
'love': 1803
'story': 1747
'find': 1604
'first': 1447


In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [64]:
# Initialize and train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train_bow, y_train)

MultinomialNB()

In [65]:
# Make predictions on the test set
y_pred = nb_model.predict(X_development_bow)


In [66]:
y_pred_train = nb_model.predict(X_train_bow)

In [67]:
# Evaluate accuracy and other metrics
accuracy = accuracy_score(y_dev, y_pred)
report = classification_report(y_dev, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)


Accuracy: 0.40274064171123
Classification Report:
               precision    recall  f1-score   support

           0       0.38      0.46      0.42       536
           1       0.62      0.03      0.06       240
           2       0.57      0.10      0.17       423
           3       0.42      0.50      0.46       416
           4       1.00      0.01      0.02        81
           5       0.67      0.06      0.11       227
           6       0.80      0.97      0.88        29
           7       0.94      0.89      0.92        19
           8       0.90      0.40      0.55        90
           9       0.36      0.69      0.48       865
          10       1.00      0.03      0.06        66

    accuracy                           0.40      2992
   macro avg       0.70      0.38      0.38      2992
weighted avg       0.50      0.40      0.35      2992



In [68]:
# Evaluate accuracy and other metrics
accuracy = accuracy_score(y_pred_train, y_train)
report = classification_report(y_pred_train, y_train)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.7671586187132827
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.74      0.82      1630
           1       0.58      1.00      0.73       312
           2       0.64      0.97      0.77       671
           3       0.77      0.68      0.72      1091
           4       0.27      1.00      0.42        45
           5       0.58      0.98      0.73       288
           6       1.00      0.93      0.97        74
           7       0.92      0.98      0.95        49
           8       0.45      0.86      0.60       103
           9       0.91      0.70      0.79      2700
          10       0.16      1.00      0.28        16

    accuracy                           0.77      6979
   macro avg       0.66      0.89      0.71      6979
weighted avg       0.82      0.77      0.78      6979



Now trying GridSearch with smoothing hyperparameter to account for over and under-fitting.

In [69]:
# performing grid search on best smoothing parameter
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]}

# Create the GridSearchCV object
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')

# Fit the model with different alpha values
grid_search.fit(X_train_bow, y_train)

# Get the best alpha value
best_alpha = grid_search.best_params_['alpha']

# Train the model with the best alpha value
nb_model_hp = MultinomialNB(alpha=best_alpha)
nb_model_hp.fit(X_train_bow, y_train)

MultinomialNB(alpha=0.1)

In [70]:
# Make predictions on the test set
y_pred = nb_model_hp.predict(X_development_bow)

In [71]:
# Evaluate accuracy and other metrics
accuracy = accuracy_score(y_dev, y_pred)
report = classification_report(y_dev, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)


Accuracy: 0.40942513368983957
Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.46      0.45       536
           1       0.40      0.15      0.21       240
           2       0.40      0.18      0.25       423
           3       0.39      0.49      0.43       416
           4       0.69      0.11      0.19        81
           5       0.43      0.22      0.29       227
           6       0.80      0.97      0.88        29
           7       0.95      0.95      0.95        19
           8       0.90      0.42      0.58        90
           9       0.37      0.60      0.46       865
          10       0.47      0.14      0.21        66

    accuracy                           0.41      2992
   macro avg       0.57      0.42      0.44      2992
weighted avg       0.43      0.41      0.39      2992



Trying more features to increase model complexity.

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, min_df=1)
training_text = training['processed_description']
development_text = development['processed_description']

X_train_bow_complex = vectorizer.fit_transform(training_text)
X_development_bow_complex = vectorizer.transform(development_text)

print('Number of unigram', len(vectorizer.get_feature_names_out()))

Number of unigram 44731


In [73]:
# performing grid search on best smoothing parameter
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]}

# Create the GridSearchCV object
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='accuracy')

# Fit the model with different alpha values
grid_search.fit(X_train_bow_complex, y_train)

# Get the best alpha value
best_alpha = grid_search.best_params_['alpha']

# Train the model with the best alpha value
nb_model_c = MultinomialNB(alpha=best_alpha)
nb_model_c.fit(X_train_bow_complex, y_train)

MultinomialNB(alpha=0.1)

In [74]:
# Make predictions on the test set
y_pred = nb_model_c.predict(X_development_bow_complex)

In [75]:
# Evaluate accuracy and other metrics
accuracy = accuracy_score(y_dev, y_pred)
report = classification_report(y_dev, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)


Accuracy: 0.4047459893048128
Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.45      0.43       536
           1       0.47      0.13      0.20       240
           2       0.43      0.17      0.24       423
           3       0.39      0.49      0.43       416
           4       0.86      0.07      0.14        81
           5       0.43      0.17      0.25       227
           6       0.80      0.97      0.88        29
           7       0.95      0.95      0.95        19
           8       0.90      0.42      0.58        90
           9       0.36      0.61      0.46       865
          10       1.00      0.11      0.19        66

    accuracy                           0.40      2992
   macro avg       0.64      0.41      0.43      2992
weighted avg       0.45      0.40      0.38      2992



In [77]:
y_pred_train = nb_model_c.predict(X_train_bow_complex)

In [78]:
# Evaluate accuracy and other metrics
accuracy = accuracy_score(y_pred_train, y_train)
report = classification_report(y_pred_train, y_train)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

Accuracy: 0.9540048717581315
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.93      0.96      1383
           1       0.99      1.00      0.99       533
           2       0.94      0.98      0.96       970
           3       0.95      0.93      0.94       984
           4       0.93      0.99      0.96       158
           5       0.98      0.99      0.98       480
           6       1.00      1.00      1.00        69
           7       0.96      1.00      0.98        50
           8       0.59      0.99      0.74       117
           9       0.96      0.94      0.95      2139
          10       0.97      0.99      0.98        96

    accuracy                           0.95      6979
   macro avg       0.93      0.98      0.95      6979
weighted avg       0.96      0.95      0.96      6979

